# Grid Placement Baseline

Implementing the grid placement strategy from Zaburo kernel as a baseline.
Target: ~88.33 score

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

# Test tree creation
tree = ChristmasTree(0, 0, 0)
print(f"Tree polygon vertices: {len(tree.polygon.exterior.coords)}")
print(f"Tree bounds: {tree.polygon.bounds}")

Tree polygon vertices: 16
Tree bounds: (-350000000000000.0, -200000000000000.0, 350000000000000.0, 800000000000000.0)


In [3]:
def check_overlap(trees):
    """Check for overlaps using ZERO tolerance.
    Returns True if any overlap exists."""
    for i in range(len(trees)):
        for j in range(i + 1, len(trees)):
            poly_i = trees[i].polygon
            poly_j = trees[j].polygon
            # Check if polygons intersect but don't just touch
            if poly_i.intersects(poly_j) and not poly_i.touches(poly_j):
                return True
    return False

def get_bounding_box_side(trees):
    """Calculate the side length of the square bounding box."""
    all_coords = []
    for t in trees:
        coords = np.asarray(t.polygon.exterior.xy).T / 1e15
        all_coords.append(coords)
    all_coords = np.concatenate(all_coords)
    
    min_x, min_y = all_coords.min(axis=0)
    max_x, max_y = all_coords.max(axis=0)
    
    return max(max_x - min_x, max_y - min_y)

print("Overlap and bounding box functions defined.")

Overlap and bounding box functions defined.


In [4]:
def find_best_trees(n: int) -> tuple:
    """Find best grid arrangement for n trees."""
    best_score, best_trees = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
    
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else Decimal("0.7") / 2
                y = r // 2 * Decimal("1.0") if r % 2 == 0 else (Decimal("0.8") + (r - 1) // 2 * Decimal("1.0"))
                row_trees = [ChristmasTree(center_x=Decimal("0.7") * i + x_offset, center_y=y, angle=angle) for i in range(m)]
                all_trees.extend(row_trees)
    
                r += 1
            
            if len(all_trees) != n:
                continue
                
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in all_trees])
    
            min_x, min_y = xys.min(axis=0)
            max_x, max_y = xys.max(axis=0)

            score = max(max_x - min_x, max_y - min_y) ** 2
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

# Test for small n
score, trees = find_best_trees(5)
print(f"N=5: score={score:.6f}, trees={len(trees)}")
print(f"Has overlap: {check_overlap(trees)}")

N=5: score=4.000000, trees=5
Has overlap: False


In [5]:
# Generate solutions for all N from 1 to 200
print("Generating solutions for N=1 to 200...")

solutions = []
for n in tqdm(range(1, 201)):
    score, trees = find_best_trees(n)
    solutions.append((score, trees))

print(f"Generated {len(solutions)} solutions")

Generating solutions for N=1 to 200...


  0%|          | 0/200 [00:00<?, ?it/s]

  6%|▌         | 12/200 [00:00<00:01, 98.29it/s]

 11%|█         | 22/200 [00:00<00:06, 28.00it/s]

 14%|█▍        | 28/200 [00:01<00:10, 16.09it/s]

...

 99%|█████████▉| 198/200 [08:02<00:14,  7.14s/it]

100%|█████████▉| 199/200 [08:10<00:07,  7.21s/it]

100%|██████████| 200/200 [08:17<00:00,  7.29s/it]

100%|██████████| 200/200 [08:17<00:00,  2.49s/it]

Generated 200 solutions


In [6]:
# Calculate overall score
overall_score = sum(score / n for n, (score, _) in enumerate(solutions, 1))
print(f"Overall score: {overall_score:.6f}")

# Show breakdown by N ranges
print("\nScore breakdown by N range:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(solutions[n-1][0] / n for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.6f}")

Overall score: 88.329998

Score breakdown by N range:
  N=1-10: 6.808653
  N=11-50: 19.360095
  N=51-100: 21.455072
  N=101-150: 20.588345
  N=151-200: 20.117833


In [7]:
# Validate no overlaps
print("Validating no overlaps...")
has_any_overlap = False
for n, (score, trees) in enumerate(solutions, 1):
    if check_overlap(trees):
        print(f"OVERLAP DETECTED at N={n}!")
        has_any_overlap = True

if not has_any_overlap:
    print("No overlaps detected in any configuration!")
else:
    print("WARNING: Overlaps detected - submission will fail!")

Validating no overlaps...


No overlaps detected in any configuration!


In [8]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n, (_, all_trees) in enumerate(solutions, 1):
    assert len(all_trees) == n, f"Expected {n} trees, got {len(all_trees)}"
    for i_t, tree in enumerate(all_trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

df = pd.DataFrame(rows)
print(f"Submission shape: {df.shape}")
print(df.head(10))

Submission shape: (20100, 4)
      id      x     y     deg
0  001_0   s0.0  s0.0    s0.0
1  002_0   s0.0  s0.0    s0.0
2  002_1  s0.35  s0.8  s180.0
3  003_0   s0.0  s0.0    s0.0
4  003_1   s0.7  s0.0    s0.0
5  003_2  s0.35  s0.8  s180.0
6  004_0   s0.0  s0.0    s0.0
7  004_1   s0.7  s0.0    s0.0
8  004_2  s0.35  s0.8  s180.0
9  004_3  s1.05  s0.8  s180.0


In [9]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
df.to_csv('/home/submission/submission.csv', index=False)
print("Saved to /home/submission/submission.csv")

# Also save to experiment folder
df.to_csv('/home/code/experiments/001_grid_baseline/submission.csv', index=False)
print("Also saved to experiment folder")

print(f"\nFinal Score: {overall_score:.6f}")

Saved to /home/submission/submission.csv
Also saved to experiment folder

Final Score: 88.329998
